In [1]:
import os
import time

import faiss
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [3]:
train_ds = load_dataset("pietrolesci/ag_news", "concat")["train"]

Reusing dataset ag_news (/home/pl487/.cache/huggingface/datasets/pietrolesci___ag_news/concat/1.0.0/5ee6e111adc7a901ca734b79fbebff09d9dba91722387a794efff8d9c178a6a3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
index = faiss.IndexFlatIP(768)
index = faiss.IndexIDMap(index)

In [5]:
count = 0
for batch in tqdm(DataLoader(train_ds["text"], batch_size=1_000)):
    ids = np.array(range(count, count + len(batch)))
    embeddings = model.encode(batch, convert_to_numpy=True).astype(np.float32)

    index.add_with_ids(embeddings, ids)
    count += len(batch)

100%|██████████| 120/120 [01:32<00:00,  1.29it/s]


In [7]:
faiss.write_index(index, "train_ag_news.faiss")

In [8]:
index = faiss.read_index("train_ag_news.faiss")

In [28]:
q = model.encode(["Italy", "Spain"]).astype(np.float32)

In [34]:
index.assign(q, 1).flatten()

array([10923, 80513])

In [47]:
i = index.index

In [51]:
a = i.reconstruct(0)

In [23]:
train_ds[33612]

{'text': 'Sito accreditato da: AMD : sicurezza e prestazioni ottimali con il nuovo processore mobile per notebook leggeri e sottili; Acer Inc. preme sull #39;acceleratore con il nuovo notebook a marchio Ferrari.',
 'label': 3}

In [65]:
remove_id = np.array([1], dtype=np.int64)
index.remove_ids(remove_id)

1

In [86]:
index.search(a.reshape(1, -1), 10)

(array([[247.47156, 199.71947, 198.06192, 197.85085, 197.11307, 196.84398,
         196.74387, 194.94043, 194.66951, 194.17671]], dtype=float32),
 array([[     9,  24571, 111312, 104201,  29212,  95747,  73158,  65840,
          73442,  62418]]))

(768,)

In [ ]:
train_ds_with_embeddings.add_faiss_index(column="embeddings")

In [ ]:
train_ds_with_embeddings.save_faiss_index("embeddings", "train_split_index.faiss")

In [ ]:
ds["train"].load_faiss_index("embeddings", "train_split_index.faiss")

In [ ]:
ds["train"].list_indexes()

In [ ]:
ds["train"].get_nearest_examples("embeddings")